In [13]:
import MDAnalysis as mda
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis as HBA
# 读取PDB文件,包含100帧
u = mda.Universe('/mnt/nas1/lanwei-125/IL8/v1/QHSHPYP/fixed.prmtop','/mnt/nas1/lanwei-125/IL8/v1/QHSHPYP/fixed.mdcrd')
hbonds = HBA(universe=u,donors_sel='resid 1:141',acceptors_sel='resid 142:149')
hbonds.run()
print(hbonds.results.hbonds)

done


In [ ]:

import os
import csv
import pandas as pd
import matplotlib.pyplot as plt

# 设置图形大小
plt.figure(figsize=(15,8)) 

hbond = '/mnt/nas1/lanwei-125/IL8/v2/QHSHPYP/nhb.dat'
# 判断xvg文件是否存在
x = []
y = []
with open(hbond,'r') as f:
    next(f) # 跳过第一行
    for line in f:
        cols = line.split()
        x.append(float(cols[0])/500) 
        y.append(float(cols[1]))

# 绘制折线图
#plt.plot(x, y, '-', label=subdir) 
#plt.plot(x, y, 'r-', label=subdir)       
plt.plot(x, y,  linewidth=0.5)

# 设置轴标签        
plt.xlabel('Time (ns)')
plt.ylabel('hbond (numbers)') 

# 设置图例
plt.legend(loc = 'upper center',bbox_to_anchor=(0.5, 1.15), ncol = 5 )

plt.show()
# 保存图像
#plt.savefig(RMSD_line,dpi = 600)